# Pinboard

Pinboard is a social bookmarking site where people share links to content and *tag* them by assigning a word that describes the content. These tags are free-form, and each user decides which ones to use.

Pinboard has a nice [API](https://pinboard.in/api/) for interacting with your own bookmarks, but not for getting all public bookmarks for a tag. Pinboard also makes all tag pages available as RSS, e.g. https://feeds.pinboard.in/rss/t:covid-19 but it unfortunately doesn't allow paging back in time.

So unfortunately we're going to have to scrape the pages. But fortunately this won't be too difficult with the [requests_html](https://requests-html.kennethreitz.org/) module because Pinboard has done such a nice job of using [semantic html](https://en.wikipedia.org/wiki/Semantic_HTML).

In [36]:
import time
import requests_html
import dateutil.parser

def pinboard(hashtag):
    http = requests_html.HTMLSession()
    pinboard_url = 'https://pinboard.in/t:{}'.format(hashtag)
    while True:
        resp = http.get(pinboard_url)
        bookmarks = resp.html.find('.bookmark')
        for b in bookmarks:
            a = b.find('.bookmark_title', first=True)
            yield {
                'url': a.attrs['href'],
                'title': a.text,
                'created': dateutil.parser.parse(b.find('.when', first=True).attrs['title'])
            }
    
        a = resp.html.find('#top_earlier', first=True)
        if not a:
            break
    
        next_url = 'https://pinboard.in' + a.attrs['href']
        if pinboard_url == next_url:
            break
        
        time.sleep(1)
        pinboard_url = next_url

In [37]:
next(pinboard('covid-19'))

{'url': 'https://www.washingtonpost.com/health/2020/03/23/coronavirus-isnt-alive-thats-why-its-so-hard-kill/',
 'title': 'The science of why coronavirus is so hard to stop - The Washington Post',
 'created': datetime.datetime(2020, 3, 23, 22, 0, 21)}

Now we can write all the results to a CSV file. But lets look for a few variants: covid-19, covid_19, covid19. To avoid repeating the same urls we can keep track of them and only write them once.

In [38]:
import csv

urls = set()
with open('data/pinboard.csv', 'w') as fh:
    out = csv.DictWriter(fh, fieldnames=['url', 'created', 'title'])
    out.writeheader()
    for hashtag in ['covid-19', 'covid_19', 'covid19']:
        for bookmark in pinboard(hashtag):
            if bookmark['url'] not in url:
                out.writerow(bookmark)
                urls.add(bookmark['url'])            

In [39]:
import pandas

# prevent dataframe columns from being truncated
pandas.set_option('display.max_columns', None)
pandas.set_option('display.width', None)
pandas.set_option('display.max_colwidth', None)

df = pandas.read_csv('data/pinboard.csv')
df

,url,created,title
0,https://www.washingtonpost.com/health/2020/03/23/coronavirus-isnt-alive-thats-why-its-so-hard-kill/,2020-03-23 22:00:21,The science of why coronavirus is so hard to stop - The Washington Post
1,https://digg.com/2020/social-distancing-coronavirus-animation,2020-03-23 21:59:05,A Brilliant Illustration Of How Social Distancing Cuts Down The Spread Of Coronavirus
2,https://www.gudethinking.com/?lightbox=dataItem-k800dsqm,2020-03-23 21:59:00,Break the chain of infection - infographic
3,https://mutualaidmamas.com/,2020-03-23 21:43:20,Welcome to Mutual Aid Medford and Somerville (MAMAS) network | Mutual Aid Medford and Somerville Network (MAMAS)
4,https://eand.co/the-upsides-of-a-global-pandemic-4dbb00be4a03,2020-03-23 21:22:24,302 Found
...,...,...,...
934,https://myeverettnews.com/2020/03/20/big-exception-list-to-everett-mayors-order-to-stay-home/,2020-03-21 06:45:36,Big Exception List To Everett Mayor's Order To Stay Home #COVID19
935,https://link.medium.com/SPeGYBeR14,2020-03-21 06:44:49,corona virus : why you must act now
936,https://twitter.com/i/web/status/1241155701822476288,2020-03-21 06:23:58,Twitter
937,https://www.thestar.com/news/gta/2020/03/20/leaked-email-reveals-ontario-regional-medical-officers-criticism-of-provincial-covid-19-strategy-as-cracks-emerge-in-front-line.html,2020-03-21 06:23:13,Leaked email reveals Ontario regional medical officer’s criticism of provincial COVID-19 strategy as cracks emerge in front line | The Star
